In [2]:
import selenium
selenium.__version__

'3.141.0'

In [ ]:
ID = ''
PASS = ''

# 목적
* 인스타그램의 셀럽 사진을 크롤링
* 입력
  * 셀럽 계정
* 출력
  * 셀럽 사진

In [ ]:
# 필수 모듈 불러오기
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## CONSTANT
* 인스타 로그인용 XPATH

In [ ]:
xpath_user_account = '//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[2]/div/label/input'
xpath_user_passwd = '//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[3]/div/label/input'
btn_login = '//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[4]/button'

## 크롬 브라우저 띄우고 인스타 접속

In [ ]:
def wait(xpath):
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_all_elements_located((By.XPATH, xpath))
        )
    finally:
        pass

In [ ]:
# 크롬 띄움
driver_path = './webdriver/chromedriver.exe'
driver = webdriver.Chrome(driver_path)

In [ ]:
# 인스타그램 메인 주소
insta_address = 'https://www.instagram.com/'

# 인스타그램 웹사이트에 접속
driver.get(insta_address)

wait(xpath_user_account)

## 로그인

In [ ]:
driver.find_element_by_xpath(xpath_user_account).send_keys(ID)
driver.find_element_by_xpath(xpath_user_passwd).send_keys(PASS)
driver.find_element_by_xpath(btn_login).click()
wait('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')

## 셀럽 주소 리스트

In [ ]:
celeb_list = ['kyliejenner']

In [ ]:
# 셀럽 인스타로 이동
driver.get(insta_address + celeb_list[0])
wait('//*[@id="react-root"]/section/main/div/header/section/div[1]/h2')

In [ ]:
# 이 값은 크롬 창 크기에 의존적임
scrollto = 400
driver.execute_script(f'window.scrollTo(0, {scrollto});')

In [ ]:
def screenshot(i, j):
    '''
    args: 
        셀럽 이름, 인스타사진 인덱스 번호
        i 는 1부터 시작
        j 는 1부터 시작 3까지
    return: 
        사진 있으면 elem, 아니면 False 리턴
    '''
    try:
        xpath = f'//*[@id="react-root"]/section/main/div/div[3]/article/div/div/div[{i}]/div[{j}]/a/div[1]/div[1]/img'
        elem = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, xpath))
        )[0]
        
        return elem
    
    except :
        return False

In [ ]:
s = time.time()

row = 1
cnt = 0

In [ ]:
while True:
    # 여러 줄의 사진이 로딩 된 이후 xpath 가 초기화 됨
    if row == 17:
        row = 1
    
    for col in range(1,4):
        elem = screenshot(row, col)
        if elem == False:
            print('break: row',row,'col',col)
            break
        elem.screenshot(f'../data/{celeb_list[0]}_{cnt}.jpg')
        
        if cnt % 10 == 0 and cnt != 0:
            print(f'{cnt}개 사진 수집 중... {time.time() - s:.2f} 초 경과')

        cnt += 1
        
    if elem == False:
        break
        
    row += 1

In [ ]:
row, cnt

## 크롬 브라우저 종료

In [ ]:
driver.quit()

## TO DO
* 셀럽 리스트에 따라 각자의 디렉토리에 사진이 저장되도록 구현해야함
* 사진 위치에 따라 잘려서 스샷 찍히거나 로딩 덜 된거 찍히는 경우 또는 인스타 로고가 함께 찍히는 경우 고쳐야함